# MATH-UA 2081.001 Final

## Nate Huang 

### Problem 1.

(a)
$$
(x^Te)^2 = |<x, e>|^2
$$
By Cauchy-Schwarz inequality, 
$$
(x^Te)^2 = |<x, e>|^2 \le ||x||^2 \cdot ||e||^2 =  n x^T x
$$


Then 
$$
c(x) = \frac{(x^Te)^2}{nx^Tx}\le 1
$$

Now look at $\hat{\rho}(x)=\frac{x^TRx}{(x^Te)^2}$.

First since $R$ is a positive-semidefinite matrix, $x^TRx\ge 0$ and $(x^Te)^2>0$ since $x\ne 0$.

So it is obvious that $\hat{\rho}(x)\ge 0$. 

Note that $R=\sum \lambda_i(v_i v_i^T)$, then $x^TRx=\sum \lambda_i(x^Tv_i)^2$.

By Parseval's indentity, $(x^Tv_i)^2 = x^Tx$. 

Therefore, $x^TRx=\sum \lambda_ix^Tx\le \lambda_n \sum x^Tx$, where $\lambda_n$ is the largest eigenvalue of $R$.

Note that $tr(R) = \sum \lambda_i = n$, then $\lambda_n\le n$. 

Therefore, $x^TRx\le nx^Tx$.

Now, 
$$
\hat{\rho}(x)=\frac{x^TRx}{(x^Te)^2}\le \frac{nx^Tx}{(x^Te)^2} = \frac{1}{c(x)}
$$


In conclusion,
$$
0\le \hat{\rho}(x)\le \frac{1}{c(x)}
$$

(b).

$$
\rho(x)=\frac{x^TRx-x^Tx}{(x^Te)^2-x^Tx} = \frac{n\frac{x^TRx}{(x^Te)^2}-\frac{nx^Tx}{(x^Te)^2}}{n-\frac{nx^Tx}{(x^Te)^2}}
$$

$\implies$
$$
\rho(x)=\frac{n\hat{\rho}(x)-\frac{1}{c(x)}}{n-\frac{1}{c(x)}} = \frac{nc(x)\hat{\rho}(x)-1}{nc(x)-1}
$$

Next

$$
\hat{\rho}(x)-\rho(x)=\hat{\rho}(x)-\frac{nc(x)\hat{\rho}(x)-1}{nc(x)-1}
=\frac{nc(x)\hat{\rho}(x)-\hat{\rho}(x)-nc(x)\hat{\rho}(x)+1}{nc(x)-1}
=\frac{1-\hat{\rho}(x)}{nc(x)-1}
$$

Since $\hat{\rho}(x)\ge 0$,
$$
\frac{1-\hat{\rho}(x)}{nc(x)-1}\le \frac{1}{nc(x)-1}
$$
and since $\hat{\rho}(x)\le \frac{1}{c(x)}$,
$$
\frac{1-\hat{\rho}(x)}{nc(x)-1}\ge \frac{1-\frac{1}{c(x)}}{nc(x)-1} = -\frac{1-c(x)}{c(x)}\frac{1}{nc(x)-1}
$$

In conclusion,
$$
-\frac{1-c(x)}{c(x)}\frac{1}{nc(x)-1}
\le \frac{1-\hat{\rho}(x)}{nc(x)-1} 
\le \frac{1}{nc(x)-1}
$$

(c)

Take the conclusion from (b)
$$
-\frac{1-c(x)}{c(x)}\frac{1}{nc(x)-1}
\le \hat{\rho}(x)-\rho(x)
\le \frac{1}{nc(x)-1}
$$

As $n\to \infty$, $\frac{1}{nc(x)-1}\to 0$, then $0\le \hat{\rho}(x)-\rho(x) \le 0 \implies \hat{\rho}(x)\sim\rho(x)$

To see the speed of convergence,
$$
\lim_{k\to \infty}\frac{\frac{1}{(k+1)c(x)-1}}{\frac{1}{kc(x)-1}}
=\lim_{k\to \infty}\frac{c(x)-\frac{1}{k}}{(1+\frac{1}{k})c(x)-\frac{1}{k}}
=1
$$

It converge sublinearly.

### Problem 2

(a)

The P&L of Vanilla Dispersion Trades is
$$
\int_{0}^{T}[\Gamma_{t, Basket}^{\$}(\sigma_{t,Basket}^2-\sigma_{Basket}^{*2})-\beta\sum_{i=1}^nw_i\Gamma_{t,i}^{\$}(\sigma_{t,i}^2-\sigma_i^{*2})]dt
$$
where 
$$
\beta \approx \frac{\sigma^{*}_{Basket}}{\sum_{i=1}^nw_i\sigma^*_i}
$$
where $\sigma^*$ is the ATM implied volatilities.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [2]:
# read and clean the data
options = pd.read_excel("2801.001.Sp20 Final_data.xlsx", sheet_name=1)
options = options.rename(columns={'Smile string: strike1; vol1; strike2; vol2;   etc. (in no particular order!)': 'IV'})

history_price = pd.read_excel("2801.001.Sp20 Final_data.xlsx", sheet_name=2)
history_price = history_price.rename(columns={'ISIN':'date'})

In [3]:
# use dictioanry to store strike price and implied vol
options['IV'] = options['IV'].str.split(';').apply(
    lambda x: {float(x[i]):float(x[i+1]) for i in range(0, len(x)-1, 2)})

In [4]:
weight = history_price.iloc[0, 1:].rename('weight')

In [5]:
options = options.merge(weight, how = 'inner', left_on='ISIN', right_on=weight.index)

Use the closet strike price from the list to estimate the implied volatility.

In [6]:
def find_closest_imp_vol(spot, imp_vol):
    d = imp_vol
    keys = np.array(list(d.keys()))
    key = keys[(np.abs(keys-spot)).argmin()]
    return d[key]

In [7]:
# calculate all the ATM implied volatitility
options["atmIV"] = options.apply(lambda x: find_closest_imp_vol(x['Spot price'], x["IV"]), axis=1)

In [8]:
beta = options.loc[0, 'atmIV']/np.sum(options.loc[1:, 'atmIV']*options.loc[1:, 'weight']/100)

In [9]:
print(f"beta is {beta}")

beta is 0.8365623281957845


In [10]:
# function that calculates dollar gamma
def gamma_dollar(S, K, r, sigma, t):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * t) / (sigma * np.sqrt(t))
    phi = np.exp(-d1*d1/2)/np.sqrt(2*np.pi)
    return S * phi / (2 * sigma * np.sqrt(t))

In [11]:
hps = history_price.iloc[1:, :].set_index('date').sort_index()

Use the ATM implied vol as an estimation of instantaneous volatility

In [12]:
IV = options[["ISIN", 'IV']].set_index('ISIN')
# calculate all the instantaneous vol
atmIV = hps.apply(lambda x: x.apply(lambda y: find_closest_imp_vol(y, IV.loc[x.name][0])), axis=0)

In [13]:
op = options.set_index('ISIN')[['Spot price', '6M forward price', 'weight', 'atmIV']]

In [14]:
# calculate all the gamma
r = options.iloc[0, 4]
lastday = hps.index[-1]
T = ((lastday - hps.index).days + 1)/365.25
gammas = hps.apply(lambda x: 
                   gamma_dollar(x, 
                                op.loc[x.name, "Spot price"], 
                                r, 
                                atmIV.loc[:, x.name], 
                                T), 
                   axis=0)

In [15]:
# instantaneous vol - ATM implied vol
var_dif = atmIV**2 - op['atmIV']**2
# gamma * above
gv = gammas * var_dif

basket_dif = gv.loc[:, "SX5E index"]
ind_dif = gv.loc[:, ~gv.columns.isin(["SX5E index"])]
# beta * above
bind_dif = beta*(ind_dif*weight).sum(axis=1)

# use dicretized sum to estimate the integral
PNL = (basket_dif - bind_dif).cumsum()

In [16]:
print(f"The daily PNL are ")
print(PNL)

The daily PNL are 
date
2011-04-29     -6.379516
2011-05-02    -13.715957
2011-05-03    -26.104407
2011-05-04    -31.616196
2011-05-05    -38.311419
                 ...    
2011-10-25   -279.883322
2011-10-26   -282.681700
2011-10-27   -282.804600
2011-10-28   -287.771791
2011-10-31   -287.943132
Length: 132, dtype: float64


In [17]:
print(f"The final PNL is {PNL[-1]}")

The final PNL is -287.9431318854586


(b)

Average correlation
$$
\rho(x) = \frac{x^TRx-x^Tx}{(x^Te)^2-x^Tx}
$$

In [18]:
ret = hps.pct_change().iloc[1:, 1:]

In [19]:
realized_cov = ret.cov()
realized_std = np.sqrt(np.diag(np.diag(realized_cov)))
std_inv = np.linalg.inv(realized_std)
realized_rho = std_inv @ np.array(realized_cov) @ std_inv

Using the equal weight. Use the proxy:
$$
\rho = \frac{2}{n(n-1)}\sum_{i<j}\rho_{i,j}
$$

In [20]:
n = realized_rho.shape[0]
rho_ij_sum = (realized_rho.sum() - n)/2
realized_avg_cor = 2*rho_ij_sum/(n*(n-1))
print(f"The realized average correlation with equal weights is {realized_avg_cor}.")

The realized average correlation with equal weights is 0.6871218531556406.


Using basket weight.

In [21]:
w = weight[1:]
realized_avg_cor = (w.T@realized_rho@w - w.T@w)/((w.T@np.ones(n))**2-w.T@w)
print(f"The realized average correlation with basket weights is {realized_avg_cor}.")

The realized average correlation with basket weights is 0.7051965960097385.


(c)

Note that 
$$
\beta \approx \sqrt{\rho_{ATMF}^{*}}
$$
Then 
$$
\rho_{ATMF}^{*} \approx \beta^2
$$

Using basket weight

In [22]:
implied_avg_cor = beta**2
print(f"The implied average correlation with basket weights is {implied_avg_cor}.")

The implied average correlation with basket weights is 0.6998365289563514.


(d)

Note that 
$$
payoff = \sigma^2_{Basket}-\beta\sum_{i=1}^{n}w_i\sigma_i^2
$$
where $\sigma$ is the realized volatility and $$\beta = \frac{\sigma_{Basket}^{\star 2}}{\sum_{i=1}^{n}w_i\sigma_i^{\star 2}}$$ where $\sigma^{\star}$ is the fair variance swap strikes.

Note that
$$
\sigma^{\star} \approx \frac{2e^{rT}}{T}[\sum_{i=1}^{n}\frac{p(K_i)}{K_i^2}\Delta K_i + \sum_{i=n+1}^{n+m}\frac{c(K_i)}{K_i^2}\Delta K_i]
$$

In [23]:
def call(S, K, tau, r, sigma):
    '''price a call option by using the BS formula'''
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))    
    price = S * norm.cdf(d1) - K * np.exp(-r * tau) * norm.cdf(d2)
    
    return price

def put(S, K, tau, r, sigma):
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))    
    price = K * np.exp(-r * tau) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    return price

In [24]:
# get the forward price
F = op["6M forward price"]

In [25]:
# get all the strikes from the IV dictionary
strikes = pd.DataFrame(IV.apply(lambda x: sorted(list(x[0].keys())), axis=1).tolist(), index = IV.index)
# get the step from strike to strike
s_diff = strikes.diff(axis=1)
# the first strike use the same deltaK as the second one
s_diff.iloc[:, 0] = s_diff.iloc[:, 1]

In [26]:
# get the spot price at the beginning
spots = history_price.iloc[-1, 1:].rename('spot')

In [27]:
# calculate the fraction (option price)/K^2
fact = strikes.apply(lambda x: 
              x.apply(lambda y: 
                      call(spots[x.name], y, 1, r, IV.loc[x.name][0][y])/(y**2) if y >F[x.name]
                      else put(spots[x.name], y, 1, r, IV.loc[x.name][0][y])/(y**2) if y<=F[x.name] 
                      else 0), axis=1)

# calculate the fair strike
fair_strikes = (fact*s_diff).sum(axis=1)*2*np.exp(r)

# calculate the beta
beta = fair_strikes["SX5E index"]/(weight[1:]*fair_strikes[1:]).sum()

In [28]:
sigma2_basket = hps.pct_change().iloc[1:, 0].std()**2
PNL = sigma2_basket - beta*(pd.Series(np.diag(realized_cov), index=realized_cov.index)*w).sum()
print(f"The final PNL is {PNL}.")

The final PNL is 0.0001980874229083672.
